## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS
import traceback
from datetime import datetime
import json
import os
from datetime import datetime
import time
import math
from IPython.display import HTML, display

gis = GIS("home")

#### Now you are ready to start!

In [2]:
parameters =   {
    "config_item": {
        "itemId": "1c6da3ab71014c1887f97644e8c9560f",
        "isLayer": True,
        "LyrTblid": 0
  }
}

#### 

In [3]:
def calc_Clarke_values(taskLyrTble, element_fields, reference_dict, fieldMapping):
    # get the fields of the target layer or table
    existing_fields = taskLyrTble.properties.fields

    # For each field in element_fields list, check if it exists in the target layer or table, ignore case
    # If it does not exist, print out the missing field
    # If it does exist, create a field definition with the name field_name_Clarke, and add the field to a list of fields to add
    fields_to_add = []
    raw_Clarke_fields_lookup = {}
    org_element_fields_lookup = {}

    # If no fieldMapping is provided, use the keys in the reference_dict as the element_fields
    if fieldMapping is None or len(fieldMapping) == 0:
        # Build the fieldMapping from the reference_dict, with the key and values as the field name
        fieldMapping = {k: k for k in reference_dict.keys()}
    else:
        # print out the fields that are excluded
        excluded_fields = [k for k in fieldMapping.keys() if k not in element_fields]
        if len(excluded_fields) > 0:
            display(HTML("<b>Exclude fields that are not in the reference table</b>: {}".format(excluded_fields)))

        # filter the fieldMapping to exlude those keys that are not in the elment_fields.
        fieldMapping = {k: v for k, v in fieldMapping.items() if k in element_fields}
        
    # filter element_fields to find out whose names are in the fieldMapping keys
    fieldMapping_keys = list(fieldMapping.keys())
    element_fields = [element_field for element_field in element_fields if element_field in fieldMapping_keys]

    display(HTML("<b>Checking if these fields exist</b>: {}".format(element_fields)))
    for reference_field_name in element_fields:
        target_field_names = []
        # get the field name in the target layer or table
        # if fieldMapping is provided, use the field name in the fieldMapping
        element_field_name_in_target = None
        if fieldMapping is not None:
            reference_field_name_lower = fieldMapping[reference_field_name].lower()
            for field in existing_fields:
                if field["name"].lower() == reference_field_name_lower:
                    element_field_name_in_target = field["name"]
        else:
            reference_field_name_lower = reference_field_name.lower()
            for field in existing_fields:
                if field["name"].lower() == reference_field_name_lower:
                    element_field_name_in_target = field["name"]

        if element_field_name_in_target == None:
            display(HTML("<b>Error</b>: Field {} doesn't have a corresponding field in the target layer/table".format(reference_field_name)))
            # remove the field from the fieldMapping
            fieldMapping.pop(reference_field_name)
            continue
        else:
            for field in existing_fields:
                if field["name"] == element_field_name_in_target:
                    if field["type"] != "esriFieldTypeDouble" and field["type"] != "esriFieldTypeInteger":
                        print("<b>Error</b>: Field {} is not a numeric field. Skpped".format(field["name"]))
                        fieldMapping.pop(reference_field_name)
                    else:
                        # print("Field {} is a numeric field".format(field["name"]))
                        target_field_names.append(field["name"])


        # Loop through target_field_names and add each field to the fields_to_add list
        for target_field_name in target_field_names:
            new_field_name = "{}_Clarke".format(target_field_name)
            # check if the new_field_name already exists in the target layer or table, ignore case
            new_field_name_lower = new_field_name.lower()
            new_field_exists = False
            for field in existing_fields:
                if field["name"].lower() == new_field_name_lower:
                    new_field_exists = True
                    raw_Clarke_fields_lookup[new_field_name] = reference_field_name
                    org_element_fields_lookup[new_field_name] = target_field_name
                    break

            # if the new field does not exist, add it to the fields_to_add list
            if not new_field_exists:
                new_field = {
                    "name": new_field_name,
                    "type": "esriFieldTypeDouble",
                    "alias": "{} Clarke".format(target_field_name),
                    "nullable": True,
                    "editable": True,
                    "visible": True
                }
                fields_to_add.append(new_field)
                raw_Clarke_fields_lookup[new_field_name] = reference_field_name
                org_element_fields_lookup[new_field_name] = target_field_name


    display(HTML("<b>Fields to add</b>: {}".format(fields_to_add)))
    display(HTML("<b>raw_Clarke_fields_lookup</b>: {}".format(raw_Clarke_fields_lookup)))

    # add the fields to the target layer or table
    if len(fields_to_add) > 0:
        add_fields_response = taskLyrTble.manager.add_to_definition({"fields": fields_to_add})
        display(HTML("<b>Add Fields Response: {}</b>".format(add_fields_response)))
    else:
        display(HTML("<b>No fields to add</b>"))

    # Calculate the Clarke values for each field in field mapping list: field_value / crustal_abundance
    display(HTML("<b>Calculating Clarke values</b>"))
    for fld in fieldMapping:
        field_name = fieldMapping[fld]
        new_field_name = "{}_Clarke".format(field_name)        

        crustal_abundance = reference_dict[fld]
        if crustal_abundance is None or crustal_abundance == 0:
            display(HTML("<b>Skip {}</b> because the reference crustal abundance is null or 0"))
            continue
        else:
            display(HTML("<b>To calculate field</b>: {}".format(new_field_name)))
            calc_sql_expresison = "ROUND({}/{}, 3)".format(field_name, crustal_abundance)
            # print("field: {} = Calc Expression: {}".format(new_field_name, calc_sql_expresison))
            calc_field_response = taskLyrTble.calculate(where="{} is null".format(new_field_name), calc_expression={"field": new_field_name, "sqlExpression" : calc_sql_expresison})

            # print("Calc Field Response: {}".format(calc_field_response))

    return fieldMapping

In [4]:
def calc_Clarke_value_Tiers(taskLyrTble, element_fields, reference_dict, fieldMapping):
# get the fields of the target layer or table
    existing_fields = taskLyrTble.properties.fields
    fields_to_add = []

    # If no fieldMapping is provided, use the keys in the reference_dict as the element_fields
    if fieldMapping is None or len(fieldMapping) == 0:
        # Build the fieldMapping from the reference_dict, with the key and values as the field name
        fieldMapping = {k: k for k in reference_dict.keys()}

    # filter element_fields to find out whose names are in the fieldMapping keys
    fieldMapping_keys = list(fieldMapping.keys())
    element_fields = [element_field for element_field in element_fields if element_field in fieldMapping_keys]

    new_field_names_to_add = []
    element_New_Field_Lookup = {}
    display(HTML("<b>Checking if the ..._Clarke_Tier fields exist</b>: {}".format(element_fields)))
    for from_field_name in element_fields:
        mapped_field_name = fieldMapping[from_field_name]
        new_field_name = "{}_Clarke_Tier".format(mapped_field_name)
        # check if it matches any names of the existing_fields
        bExists = False
        for field in existing_fields:
            if field["name"].lower() == new_field_name.lower():
                bExists = True
                element_New_Field_Lookup[from_field_name] = field["name"]
                break

        if not bExists:
            new_field_names_to_add.append(new_field_name)
            element_New_Field_Lookup[from_field_name] = new_field_name

    # Add the fields to the target layer or table. The field type is integer
    if len(new_field_names_to_add) == 0:
        display(HTML("<b>No ..._Clarke_Tier fields to add</b>"))
    else:
        display(HTML("<b>Fields to add</b>: {}".format(new_field_names_to_add)))
        for new_field_name in new_field_names_to_add:
            new_field = {
                "name": new_field_name,
                "type": "esriFieldTypeInteger",
                "alias": new_field_name.replace("_", " "),
                "nullable": True,
                "editable": True,
                "visible": True,
                'domain': {
                    'type': 'codedValue',
                    'name': '{}_Domain'.format(new_field_name),
                    'codedValues': [
                        {
                            'name': 'L0 - Less than background',
                            'code': 0
                        },
                        {
                            'name': 'L1 - Background',
                            'code': 1
                        },
                        {
                            'name': 'L2 - 2-3x background',
                            'code': 2
                        },
                        {
                            'name': 'L3 - 4-7x background',
                            'code': 3
                        },
                        {
                            'name': 'L4 - 8-15x background',
                            'code': 4
                        },
                        {
                            'name': 'L5 - >15x background',
                            'code': 5
                        }
                    ]
                }
            }
            fields_to_add.append(new_field)

        add_fields_response = taskLyrTble.manager.add_to_definition({"fields": fields_to_add})
        display(HTML("<b>Add Fields Response</b>: {}".format(add_fields_response)))

    # Calculate the Tier values for each new field in element_New_Field_Lookup
    display(HTML("<b>To calculate  Tier values</b>"))
    for fld in element_New_Field_Lookup:
        new_field_name = element_New_Field_Lookup[fld]
        clarke_field_name = "{}_Clarke".format(fieldMapping[fld])
        display(HTML("<b>To calculate field</b>: {}".format(new_field_name)))
        # Here is the logic to calculate the Tier value based on the _Clarke value
        # if the value is null, set the Tier value to null
        # if the value is less than 0.5, set the Tier value to 0
        # if the value is less than 1.5, set the Tier value to 1
        # if the value is less than 3.5, set the Tier value to 2
        # if the value is less than 7.5, set the Tier value to 3
        # if the value is less than 15.5, set the Tier value to 4
        # else, set the Tier value to 5
        # The Tier value is an integer field
        # Write the above logic in SQL expression
        calc_sql_expresison = "CASE WHEN {} IS NULL THEN 0 WHEN {} < 0.5 THEN 0 WHEN {} < 1.5 THEN 1 WHEN {} < 3.5 THEN 2 WHEN {} < 7.5 THEN 3 WHEN {} < 15.5 THEN 4 ELSE 5 END".format(clarke_field_name, clarke_field_name, clarke_field_name, clarke_field_name, clarke_field_name, clarke_field_name)

        # print("field: {} = Calc Expression: {}".format(new_field_name, calc_sql_expresison))
        calc_field_response = taskLyrTble.calculate(where="1=1", calc_expression={"field": new_field_name, "sqlExpression" : calc_sql_expresison})

        # print("Calc Field Response: {}".format(calc_field_response))


In [5]:
def copyInput(taskItem, taskItemIsLayer, taskLyrTblId):
    display(HTML("<b>Copy the input {}: {}</b> ...".format("layer" if taskItemIsLayer else "table", taskItem.id)))    

    # cloned_items = gis.content.clone_items(
    #     items=[taskItem], search_existing_items=False
    # )
    username = gis.users.me.username
    if "@" in username:
        username = username.split("@")[0]

    #newItemName = "{}_{}".format(taskItem.name, username)
    newItemName = "{}_{}".format(taskItem.name, 'copy')

    result_item = taskItem.copy_feature_layer_collection(
        service_name=newItemName,
        layers=[taskLyrTblId] if taskItemIsLayer else [],
        tables=[taskLyrTblId] if not taskItemIsLayer else [],
    )

    display(HTML("<b>Copied item id</b>: <a href='https://usgs.maps.arcgis.com/home/item.html?id={}' target='_blank'>{}</a></b>".format(result_item.id, result_item.id)))
    if taskItemIsLayer:
        taskLyrTble = result_item.layers[0]
        sourceLyrTble = taskItem.layers[taskLyrTblId]
    else:
        taskLyrTble = result_item.tables[0]
        sourceLyrTble = taskItem.tables[taskLyrTblId]

    # Read the data from the source and append to the target
    source_resp = sourceLyrTble.query(
        where="1=1",
        out_fields=["*"],
        return_all_records=True,
        return_geometry=taskItemIsLayer,
    )
    source_features = source_resp.features
        
    display(HTML("Number of records in the source: {}".format(len(source_features))))
    # split the source features into batches of 2000
    # Append the source features to the target
    batch_size = 1000
    for i in range(0, len(source_features), batch_size):
        display(HTML("Appending features from {} to {}".format(i, i + batch_size)))
        
        chunk = source_features[i : i + batch_size]
        add_response = taskLyrTble.edit_features(adds=chunk)
        failed_records = list(
            filter(lambda d: d["success"] == False, add_response["addResults"])
        )
        b_Error = len(failed_records) > 0
        if b_Error:
            display(HTML("Failed records: {}".format(failed_records)))
            
            raise Exception("Failed to append features: {}".format(failed_records))

    return result_item, taskLyrTble


In [6]:
def updateItemProperties(result_item, task, gis, completedOn):
    # check if you have the right to update the item
    try:
        me = gis.users.me
        userFullName = me.fullName
        newDescription = "<B>Clarke Values calculated by: {} on {} using the following parameters:</b>  \n<br>".format(
            userFullName, completedOn
        )
        newDescription += "<ul><li>Geochemical Analysis item {} {} {}</li>".format(
            task["itemId"],
            "Layer" if task["isLayer"] else "Table",
            task["LyrTblid"],
        )
        newDescription += "<li>Clarke reference item {} Table {}</li>".format(
            task["clarke_reference_config"]["itemId"],
            task["clarke_reference_config"]["tableId"],
        )
        newDescription += "<li>Output option: {}</li>".format(task["output_option"])
        newDescription += "<li>fieldMapping: {}</li>".format(
            task["fieldMapping"]
            if task["fieldMapping"] is not None and len(task["fieldMapping"]) > 0
            else ""
        )
        newDescription += "</ul>"
        if result_item.description is not None:
            newDescription += "\n\n<br><br>" + result_item.description
        props = {"description": newDescription}

        # Check if the tags "geochemistry" and "crustal abundance" are already in the item
        # if not, add them
        tags = result_item.tags
        if "geochemistry" not in tags:
            tags.append("geochemistry")
        if "crustal abundance" not in tags:
            tags.append("crustal abundance")

        if len(tags) > 0:
            props["tags"] = tags
            
        update_response = result_item.update(item_properties=props)
        display(
            HTML(
                "<b>Update result item metadata</b>: {}".format(
                    "Successful" if update_response else "Failed"
                )
            )
        )
    except Exception as e:
        if e.args[0].find("403") > 0 and e.args[0].find("permissions") > 0:
            print("User does not have permissions to update the metadata of the item")
        else:
            print("Error updating item description: {}".format(e))


In [7]:
# Get Start Time
start_time = time.time()

try:
    # Query the config table, and build the parameters for the calculation
    config_item = parameters['config_item']
    config_table_itemId = config_item['itemId']
    config_table_item = gis.content.get(config_table_itemId)
    isLayer = config_item['isLayer']
    LyrTblid = config_item['LyrTblid']
    config_lyr_tbl = None
    if isLayer:
        config_lyr_tbl = config_table_item.layers[LyrTblid]
    else:
        config_lyr_tbl = config_table_item.tables[LyrTblid]


    config_resp = config_lyr_tbl.query(where = "skip = null or skip = 'No' ", out_fields=["*"], return_all_records = True, return_geometry=False)

    if len(config_resp.features) == 0:
        print("No configuration to process")
        exit()
    else:
        parameters['tasks'] = []
        # loop through the features and build the parameters
        for f in config_resp.features:
            task = {}
            task["clarke_reference_config"] = {
                "itemId": f.attributes["reference_item_id"],
                "tableId": f.attributes["reference_table_id"]
            }
            task["name"] = f.attributes["name"]
            task["itemId"] = f.attributes["item_id"]
            task["isLayer"] = f.attributes["layer_or_table"] == "Layer"
            task["LyrTblid"] = f.attributes["layer_or_table_id"]
            task["fieldMapping"] = f.attributes["field_mapping"]
            task["output_option"] = f.attributes["output_option"]
            task["objectid"] = f.attributes["objectid"]
            task["run_results"] = f.attributes["run_results"]
            parameters['tasks'].append(task)

        tasks = parameters['tasks']
        for task in tasks:
            display(HTML("\n\nTo run the config: {}\n".format(task["name"])))

            # get the layers and tables
            referenceItemId = task['clarke_reference_config']["itemId"]
            referenceTableId = task['clarke_reference_config']["tableId"]

            referenceItemItem=gis.content.get(referenceItemId)
            referenceTable = referenceItemItem.tables[referenceTableId]

            # Query the reference table and get the reference values
            reference_resp = referenceTable.query(where = "1=1", out_fields=["field", "crustal_abundance"], return_all_records = True, return_geometry=False)

            # build the lookup from field to the crustal abundance
            reference_dict = {f.attributes["field"]: f.attributes["crustal_abundance"] for f in reference_resp.features}

            # build the list of fields
            element_fields = list(reference_dict.keys())

            taskItemId = task["itemId"]
            taskItemIsLayer = task["isLayer"]
            taskLyrTblId = task["LyrTblid"]

            taskItem=gis.content.get(taskItemId)
            taskLyrTble = None

            result_item = taskItem
            if task["output_option"] is not None and task["output_option"] == "Create new":
                result_item, taskLyrTble = copyInput(taskItem, taskItemIsLayer, taskLyrTblId)
            else:
                if taskItemIsLayer:
                    taskLyrTble = taskItem.layers[taskLyrTblId]
                else:
                    taskLyrTble = taskItem.tables[taskLyrTblId]                
                    
            fieldMapping = {}
            if "fieldMapping" in task:
                s_fieldMapping = task["fieldMapping"]
                if s_fieldMapping is not None and len(s_fieldMapping) > 0:
                    fieldMapping = json.loads(s_fieldMapping)

            #delete_clarke_fields(taskLyrTble)

            updated_fieldMapping = calc_Clarke_values(taskLyrTble, element_fields, reference_dict, fieldMapping)
            calc_Clarke_value_Tiers(taskLyrTble, element_fields, reference_dict, updated_fieldMapping)
            #calc_Clarke_value_percentiles_nthStdDev(taskLyrTble, taskItem, element_fields, reference_dict, fieldMapping)

            display(HTML("<b>Calculation completed for the config</b>: {}".format(task["name"])))
            
            completedOn = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            updateItemProperties(result_item, task, gis, completedOn)
                        
            result_links = "<a href='https://usgs.maps.arcgis.com/home/item.html?id={}#data' target='_blank'> {}: Result of {}</a>".format(
                result_item.id,
                result_item.title,
                completedOn
            )

            display(HTML("<b>Result link: {}</b>".format(result_links)))
            if task["run_results"] is not None and len(task["run_results"]) > 0:
                result_links = task["run_results"] + "<br>\n" + result_links
                display(HTML("<b>Link to all results: <a href='https://usgs.maps.arcgis.com/home/item.html?id={}#data' target='_blank'>All results</a></b>".format(parameters["config_item"]["itemId"])))
                
                
            # save the result link to config_lyr_tbl
            edit_resp = config_lyr_tbl.edit_features(
                updates=[
                    {
                        "attributes": {
                            "run_results": result_links,
                            "objectid": task["objectid"],
                        }
                    }
                ],
                use_global_ids=False
            )
            # check if there are any errors in the edit response
            if "error" in edit_resp["updateResults"][0]:
                raise Exception(
                    "Failed to update the results link: {}".format(
                        edit_resp["updateResults"][0]
                    )
                )
       


except Exception:
    print(traceback.format_exc())

finally:
    # Log Run Time
    print('\n\nProgram Run Time: {0} Minutes'.format(round(((time.time() - start_time) / 60), 2)))




Program Run Time: 0.57 Minutes
